In [ ]:
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

**Đến trang chủ**

In [ ]:
url = 'https://vnexpress.net'
driver.get(url)

driver.execute_script(
    'window.scrollTo({top: document.body.scrollHeight, behavior: "smooth"})')
time.sleep(5)
driver.execute_script('window.scrollTo({top: 0, behavior: "smooth"})')

**Lấy link đến những bài viết**

In [ ]:
POST_LINK_CLS = '.title-news a:not(.count_cmt)'

post_links_el = driver.find_elements(By.CSS_SELECTOR, POST_LINK_CLS)
post_links = [
    el.get_attribute('href')
    for el in post_links_el]
print('Found', len(post_links), 'links.')
post_links

> Nhận xét: link của các post có những link không ở trang vnexpress, cần xử lý thêm.

In [ ]:
valid_post_links = []
VNE_REGEX = '^https://vnexpress.net'
for post_link in post_links:
    if re.match(VNE_REGEX, post_link):
        valid_post_links.append(post_link)
    else:
        print('Invalid link:', post_link)

print('Total valid links:', len(valid_post_links))

**Đi đến tất cả các trang và lấy tiêu đề + nội dung + hình thumbnail**

Yêu cầu: tiêu đề và hình

In [ ]:
import os
import re
from urllib import request
from tqdm import tqdm
from string import punctuation

In [ ]:
POST_TITLE_CLS = '.sidebar-1 .title-detail'
POST_CONTENT_CLS = 'article.fck_detail p.Normal'
POST_IMG_CLS = '.fig-picture img[itemprop="contentUrl"]'
IMAGE_REGEX = r'([\w-]+(?:.jpg|.png|.webm))'

for post_link in tqdm(valid_post_links):
    # Go to that link.
    driver.get(post_link)
    time.sleep(2)
    # Get the things we need.
    # Post title.
    title = driver.find_element(By.CSS_SELECTOR, POST_TITLE_CLS)
    # Post content.
    contents_el = driver.find_elements(By.CSS_SELECTOR, POST_CONTENT_CLS)
    contents = [content_el.get_attribute(
        'innerText') for content_el in contents_el]
    # Post images.
    images_el = driver.find_elements(By.CSS_SELECTOR, POST_IMG_CLS)
    images_src = [image_el.get_attribute('src') for image_el in images_el]
    # Save images to local.
    image_file_path = '/Users/minhquang/Desktop/ml/ml-teach/lesson_5/img'

    for image_src in images_src:
        image_file_name = re.search(IMAGE_REGEX, image_src)
        if image_file_name:
            image_file_name = image_file_name.group()
            request.urlretrieve(image_src, os.path.join(
                image_file_path, image_file_name))
            print(f'Saved', image_file_name)

    break

In [ ]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
POST_TITLE_CLS = '.title-detail'
POST_CONTENT_CLS = 'article.fck_detail p.Normal'
POST_IMG_CLS = 'picture img[itemprop="contentUrl"]'
IMAGE_URL_REGEX = r'https:\/\/[\w\-\./]+(?:.jpg|.png|.webm)'
IMAGE_REGEX = r'([\w-]+(?:.jpg|.png|.webm))'


def title_to_slug(title):
    # Remove punctuations.
    title_ = re.sub(rf'[{punctuation}]', '', title)
    # Replace duplicated spaces with one space.
    title_ = re.sub(r'\s+', ' ', title_)
    title_slug = '-'.join(title_.lower().split(' '))
    return title_slug


for post_link in valid_post_links:
    try:
        # Go to that link.
        driver.get(post_link)
        driver.maximize_window()
        driver.implicitly_wait(5)

        # Scroll to end of page.
        driver.execute_script(
            'window.scrollTo({top: document.body.scrollHeight, behavior: "smooth"})')
        # driver.execute_script('window.scrollTo({top: 0, behavior: "smooth"})')

        # Get the things we need.
        # Post title.
        title = driver.find_element(
            By.CSS_SELECTOR, POST_TITLE_CLS).get_attribute('innerText')
        title_slug = title_to_slug(title)

        # Post content.
        contents_el = driver.find_elements(By.CSS_SELECTOR, POST_CONTENT_CLS)
        contents = [content_el.get_attribute(
            'innerText') for content_el in contents_el]

        # Post images.
        images_el = driver.find_elements(By.CSS_SELECTOR, POST_IMG_CLS)
        # images_el = [image for image in WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, POST_IMG_CLS)))]
        # driver.execute_script('arguments[0].scrollIntoView({block: "center", behavior: "smooth"});', images_el[-1])
        images_src = list(set([image_el.get_attribute('src')
                        for image_el in images_el]))
        print('Found', len(images_src), 'images.')
        # Save images to local.
        image_file_path = '/Users/minhquang/Desktop/ml/ml-teach/lesson_5/img'

        for i, image_src in enumerate(images_src):
            image_file_name = re.search(IMAGE_REGEX, image_src)
            if image_file_name:
                image_file_name = image_file_name.group()
                image_save_path = os.path.join(
                    image_file_path, image_file_name)
                print(f'Saving', image_src)
                _, res = request.urlretrieve(image_src, image_save_path)
                # Get image format (png, jpg, ...).
                image_type = res.get_content_subtype()
                # Rename the saved file to correct format.
                new_image_save_path = os.path.join(
                    image_file_path, title_slug + '-' + str(i) + f'.{image_type}')
                os.rename(image_save_path, new_image_save_path)
    except Exception as e:
        print('Error ====>' e)

In [ ]:
from multiprocessing import Pool

# TODO: implement save image using multiprocessing starmap.
# May need this to do multiprocessing on urlretrieve: https://stackoverflow.com/questions/49022363/request-urlretrieve-in-multiprocessing-python-gets-stuck


def save_image_multi(image_src, index, title_slug):
    image_file_name = re.search(IMAGE_REGEX, image_src)

    if image_file_name:
        image_file_name = image_file_name.group()
        image_save_path = os.path.join(
            image_file_path, image_file_name)
        print(f'Saving', image_src)
        _, res = request.urlretrieve(image_src, image_save_path)
        # Get image format (png, jpg, ...).
        image_type = res.get_content_subtype()
        # Rename the saved file to correct format.
        new_image_save_path = os.path.join(
            image_file_path, title_slug + '-' + str(index) + f'.{image_type}')
        os.rename(image_save_path, new_image_save_path)